## Mise à Jour de la synthèse des connaissances sur les changements climatiques au Québec

### 1 Températures
#### Data: AHCCD

In [1]:
import warnings
import xarray as xr
from clisops.core import subset
from dask.diagnostics import ProgressBar
#from siphon.catalog import TDSCatalog

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
import geopandas as gpd
import matplotlib.pyplot as plt

%matplotlib inline

warnings.simplefilter("ignore")

/exec/braun/.conda/envs/obsflow/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.2-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
# might read from file ..., but not what we want!
dsa = xr.open_dataset('/tank/scenario/netcdf/ec/ahccd/gen3/ahccd_gen3_tasmax.nc')
#dsa

In [49]:
from pathlib import Path
import xscen as xs

# dcat = xs.DataCatalog('/tank/scenario/catalogues/station-obs.json') # quick version, without xscen
wdir = Path('/home/braun/python/obsflow/')
xs.load_config(str(wdir/'paths_obs.yml'), str(wdir/'config_obs.yml'), verbose=(__name__ == "__main__"), reset=True)
obs_cat = xs.CONFIG['extract']['station-obs']['search_data_catalogs']['data_catalogs']
dcat = xs.DataCatalog(*obs_cat) # DataCatalog doesn't unwrap a PosixPath object!

output_folder = Path('/scen3/braun/data/obsflow')
#output_folder.mkdir(exist_ok=True)
#cat_folder = Path('/tank/scenario/catalogues/')
#obs_cat = cat_folder / 'station-obs.json' #'obs-extra.json'

dcat.df

2023-05-04 18:12:28 INFO     xscen.config    Updated the config with /home/braun/python/obsflow/paths_obs.yml.
2023-05-04 18:12:28 INFO     xscen.config    Updated the config with /home/braun/python/obsflow/config_obs.yml.


,id,type,processing_level,bias_adjust_institution,bias_adjust_project,mip_era,activity,driving_institution,driving_model,institution,...,member,xrfreq,frequency,variable,domain,date_start,date_end,version,format,path
0,GouvCan_CanSWE_CAN,station-obs,raw,NaN,NaN,NaN,NaN,NaN,NaN,GouvCan,...,NaN,D,day,"(snd,)",CAN,1928-01-01 00:00,2020-07-31 00:00,v4,nc,/tank/scenario/datasets/station-obs/GouvCan/Ca...
1,GouvCan_CanSWE_CAN,station-obs,raw,NaN,NaN,NaN,NaN,NaN,NaN,GouvCan,...,NaN,D,day,"(snw,)",CAN,1928-01-01 00:00,2020-07-31 00:00,v4,nc,/tank/scenario/datasets/station-obs/GouvCan/Ca...
2,GouvCan_CanSWE_CAN,station-obs,raw,NaN,NaN,NaN,NaN,NaN,NaN,GouvCan,...,NaN,D,day,"(sd,)",CAN,1928-01-01 00:00,2020-07-31 00:00,v4,nc,/tank/scenario/datasets/station-obs/GouvCan/Ca...
3,GouvCan_AHCCD_CAN,station-obs,raw,NaN,NaN,NaN,NaN,NaN,NaN,GouvCan,...,NaN,D,day,"(tasmin,)",CAN,1840-03-01 00:00,2021-12-31 00:00,gen3,nc,/tank/scenario/datasets/station-obs/GouvCan/AH...
4,GouvCan_AHCCD_CAN,station-obs,raw,NaN,NaN,NaN,NaN,NaN,NaN,GouvCan,...,NaN,D,day,"(tas,)",CAN,1840-03-01 00:00,2021-12-31 00:00,gen3,nc,/tank/scenario/datasets/station-obs/GouvCan/AH...
5,GouvCan_AHCCD_CAN,station-obs,raw,NaN,NaN,NaN,NaN,NaN,NaN,GouvCan,...,NaN,D,day,"(tasmax,)",CAN,1840-03-01 00:00,2021-12-31 00:00,gen3,nc,/tank/scenario/datasets/station-obs/GouvCan/AH...
6,GouvCan_AHCCD_CAN,station-obs,raw,NaN,NaN,NaN,NaN,NaN,NaN,GouvCan,...,NaN,D,day,"(pr,)",CAN,1840-01-01 00:00,2017-12-31 00:00,gen2,nc,/tank/scenario/datasets/station-obs/GouvCan/AH...
7,GouvCan_AHCCD_CAN,station-obs,raw,NaN,NaN,NaN,NaN,NaN,NaN,GouvCan,...,NaN,D,day,"(prlp,)",CAN,1840-01-01 00:00,2017-12-31 00:00,gen2,nc,/tank/scenario/datasets/station-obs/GouvCan/AH...
8,GouvCan_AHCCD_CAN,station-obs,raw,NaN,NaN,NaN,NaN,NaN,NaN,GouvCan,...,NaN,D,day,"(prsn,)",CAN,1840-01-01 00:00,2017-12-31 00:00,gen2,nc,/tank/scenario/datasets/station-obs/GouvCan/AH...
9,GouvQc_info-climat_QC,station-obs,raw,NaN,NaN,NaN,NaN,NaN,NaN,GouvQc,...,NaN,2W,2sem,"(snw,)",QC,1928-03-19 00:00,2021-05-11 00:00,v2021,nc,/tank/scenario/datasets/station-obs/GouvQc/inf...


In [50]:
xs.CONFIG

{'paths': {'project_catalog': '/scen3/braun/data/obs_synthese23/pcat_obs.json',
  'task': '/scen3/braun/data/obs_synthese23/{processing_level}/{id}_{xrfreq}_{processing_level}.zarr',
  'ba': '/scen3/braun/data/obs_synthese23/{processing_level}/{id}_{xrfreq}_{processing_level}_{variable}.zarr'},
 'extract': {'reconstruction': {'search_data_catalogs': {'data_catalogs': ['/tank/scenario/catalogues/reconstruction-extra.json',
     '/tank/scenario/catalogues/reconstruction.json'],
    'allow_resampling': False,
    'allow_conversion': True,
    'variables_and_freqs': {'tasmax': 'D', 'tasmin': 'D', 'pr': 'D'},
    'periods': None,
    'other_search_criteria': {'source': ['ERA5-Land', 'RDRS']}},
   'extract_dataset': {'region': {'name': 'Quebec_test',
     'method': 'shape',
     'tile_buffer': 1.5,
     'shape': '/scen3/braun/data/obs_synthese23/gis/lpr_000b16a_e_QC_simpl1.zip!lpr_000b16a_e_QC_simpl1.shp'}},
   'stack_drop_nans': True,
   'dask': {'n_workers': 2, 'threads_per_worker': 3, 'me

In [51]:
for var in  ['tasmin', 'tasmax', 'pr']:
    cat = dcat.search(**{'source': 'AHCCD', 'variable': var, 'xrfreq': 'D'})
    display(cat.df)
    ds_dict = xs.extract_dataset(catalog=cat, variables_and_freqs={var:'D'})
    display(ds_dict['D'])

,id,type,processing_level,bias_adjust_institution,bias_adjust_project,mip_era,activity,driving_institution,driving_model,institution,...,member,xrfreq,frequency,variable,domain,date_start,date_end,version,format,path
0,GouvCan_AHCCD_CAN,station-obs,raw,NaN,NaN,NaN,NaN,NaN,NaN,GouvCan,...,NaN,D,day,"(tasmin,)",CAN,1840-03-01 00:00,2021-12-31 00:00,gen3,nc,/tank/scenario/datasets/station-obs/GouvCan/AH...


<xarray.Dataset>
Dimensions:       (time: 66415, station: 779)
Coordinates: (12/16)
  * time          (time) datetime64[ns] 1840-03-01 1840-03-02 ... 2021-12-31
  * station       (station) object '1096631' '7047250' ... '1108446' '7015730'
    stnid         (station) object dask.array<chunksize=(779,), meta=np.ndarray>
    station_name  (station) object dask.array<chunksize=(779,), meta=np.ndarray>
    lon           (station) float64 dask.array<chunksize=(779,), meta=np.ndarray>
    lat           (station) float64 dask.array<chunksize=(779,), meta=np.ndarray>
    ...            ...
    frommonth     (station) int32 dask.array<chunksize=(779,), meta=np.ndarray>
    toyear        (station) int32 dask.array<chunksize=(779,), meta=np.ndarray>
    tomonth       (station) int32 dask.array<chunksize=(779,), meta=np.ndarray>
    pct_miss      (station) float64 dask.array<chunksize=(779,), meta=np.ndarray>
    joined        (station) object dask.array<chunksize=(779,), meta=np.ndarray>
    rcs           (station) object dask.array<chunksize=(779,), meta=np.ndarray>
Data variables:
    tasmin        (station, time) float64 dask.array<chunksize=(779, 66415), meta=np.ndarray>
Attributes: (12/26)
    title:                   Third Generation of Homogenized Daily Temperatur...
    history:                 2022-09-27 : Convert from original format to NetCDF
    type:                    station_obs
    institute:               Environment and Climate Change Canada
    institute_id:            ECCC
    dataset_id:              AHCCD_gen3_day_temperature
    ...                      ...
    cat:date_start:          1840-03-01 00:00
    cat:date_end:            2021-12-31 00:00
    cat:version:             gen3
    cat:_data_format_:       nc
    cat:path:                /tank/scenario/datasets/station-obs/GouvCan/AHCC...
    intake_esm_dataset_key:  GouvCan_AHCCD_CAN.CAN.raw.D

,id,type,processing_level,bias_adjust_institution,bias_adjust_project,mip_era,activity,driving_institution,driving_model,institution,...,member,xrfreq,frequency,variable,domain,date_start,date_end,version,format,path
0,GouvCan_AHCCD_CAN,station-obs,raw,NaN,NaN,NaN,NaN,NaN,NaN,GouvCan,...,NaN,D,day,"(tasmax,)",CAN,1840-03-01 00:00,2021-12-31 00:00,gen3,nc,/tank/scenario/datasets/station-obs/GouvCan/AH...


<xarray.Dataset>
Dimensions:       (time: 66415, station: 779)
Coordinates: (12/16)
  * time          (time) datetime64[ns] 1840-03-01 1840-03-02 ... 2021-12-31
  * station       (station) object '3016124' '7016902' ... '6073980' '7014290'
    stnid         (station) object dask.array<chunksize=(779,), meta=np.ndarray>
    station_name  (station) object dask.array<chunksize=(779,), meta=np.ndarray>
    lon           (station) float64 dask.array<chunksize=(779,), meta=np.ndarray>
    lat           (station) float64 dask.array<chunksize=(779,), meta=np.ndarray>
    ...            ...
    frommonth     (station) int32 dask.array<chunksize=(779,), meta=np.ndarray>
    toyear        (station) int32 dask.array<chunksize=(779,), meta=np.ndarray>
    tomonth       (station) int32 dask.array<chunksize=(779,), meta=np.ndarray>
    pct_miss      (station) float64 dask.array<chunksize=(779,), meta=np.ndarray>
    joined        (station) object dask.array<chunksize=(779,), meta=np.ndarray>
    rcs           (station) object dask.array<chunksize=(779,), meta=np.ndarray>
Data variables:
    tasmax        (station, time) float64 dask.array<chunksize=(779, 66415), meta=np.ndarray>
Attributes: (12/26)
    title:                   Third Generation of Homogenized Daily Temperatur...
    history:                 2022-09-27 : Convert from original format to NetCDF
    type:                    station_obs
    institute:               Environment and Climate Change Canada
    institute_id:            ECCC
    dataset_id:              AHCCD_gen3_day_temperature
    ...                      ...
    cat:date_start:          1840-03-01 00:00
    cat:date_end:            2021-12-31 00:00
    cat:version:             gen3
    cat:_data_format_:       nc
    cat:path:                /tank/scenario/datasets/station-obs/GouvCan/AHCC...
    intake_esm_dataset_key:  GouvCan_AHCCD_CAN.CAN.raw.D

,id,type,processing_level,bias_adjust_institution,bias_adjust_project,mip_era,activity,driving_institution,driving_model,institution,...,member,xrfreq,frequency,variable,domain,date_start,date_end,version,format,path
0,GouvCan_AHCCD_CAN,station-obs,raw,NaN,NaN,NaN,NaN,NaN,NaN,GouvCan,...,NaN,D,day,"(pr,)",CAN,1840-01-01 00:00,2017-12-31 00:00,gen2,nc,/tank/scenario/datasets/station-obs/GouvCan/AH...


<xarray.Dataset>
Dimensions:       (time: 65014, station: 463)
Coordinates: (12/13)
  * time          (time) datetime64[ns] 1840-01-01 1840-01-02 ... 2017-12-31
  * station       (station) object '5030203' '7043540' ... '5062922' '8502591'
    station_name  (station) object dask.array<chunksize=(463,), meta=np.ndarray>
    stnid         (station) object dask.array<chunksize=(463,), meta=np.ndarray>
    lon           (station) float64 dask.array<chunksize=(463,), meta=np.ndarray>
    lat           (station) float64 dask.array<chunksize=(463,), meta=np.ndarray>
    ...            ...
    prov          (station) object dask.array<chunksize=(463,), meta=np.ndarray>
    fromyear      (station) int32 dask.array<chunksize=(463,), meta=np.ndarray>
    frommonth     (station) int32 dask.array<chunksize=(463,), meta=np.ndarray>
    toyear        (station) int32 dask.array<chunksize=(463,), meta=np.ndarray>
    tomonth       (station) int32 dask.array<chunksize=(463,), meta=np.ndarray>
    stns_joined   (station) object dask.array<chunksize=(463,), meta=np.ndarray>
Data variables:
    pr            (station, time) float64 dask.array<chunksize=(463, 65014), meta=np.ndarray>
Attributes: (12/26)
    title:                   Second Generation of Daily Adjusted Precipitatio...
    history:                 2022-09-27 : Convert from original format to NetCDF
    type:                    station_obs
    institute:               Environment and Climate Change Canada
    institute_id:            ECCC
    dataset_id:              AHCCD_gen2_day_precipitation
    ...                      ...
    cat:date_start:          1840-01-01 00:00
    cat:date_end:            2017-12-31 00:00
    cat:version:             gen2
    cat:_data_format_:       nc
    cat:path:                /tank/scenario/datasets/station-obs/GouvCan/AHCC...
    intake_esm_dataset_key:  GouvCan_AHCCD_CAN.CAN.raw.D

In [54]:
ds_dict['D'].prov.values

array(['MB', 'QC', 'QC', 'BC', 'BC', 'BC', 'AB', 'NS', 'NU', 'MB', 'QC',
       'QC', 'SK', 'AB', 'NB', 'ON', 'ON', 'BC', 'SK', 'BC', 'QC', 'NS',
       'ON', 'BC', 'QC', 'ON', 'BC', 'SK', 'AB', 'BC', 'ON', 'BC', 'BC',
       'SK', 'QC', 'NT', 'QC', 'NU', 'QC', 'ON', 'ON', 'NS', 'ON', 'AB',
       'NT', 'ON', 'AB', 'BC', 'QC', 'YK', 'ON', 'BC', 'NS', 'AB', 'MB',
       'BC', 'BC', 'SK', 'NB', 'QC', 'AB', 'SK', 'NU', 'YK', 'NS', 'ON',
       'NS', 'ON', 'BC', 'NT', 'NU', 'AB', 'NS', 'ON', 'ON', 'AB', 'ON',
       'BC', 'ON', 'MB', 'ON', 'NU', 'SK', 'QC', 'NU', 'SK', 'ON', 'BC',
       'QC', 'SK', 'NL', 'SK', 'SK', 'YK', 'BC', 'BC', 'SK', 'NU', 'AB',
       'BC', 'BC', 'SK', 'BC', 'ON', 'NU', 'AB', 'BC', 'NS', 'QC', 'ON',
       'SK', 'BC', 'BC', 'NU', 'BC', 'NL', 'QC', 'BC', 'NB', 'YK', 'BC',
       'NB', 'NL', 'AB', 'ON', 'BC', 'NU', 'NT', 'AB', 'ON', 'NB', 'NL',
       'QC', 'ON', 'ON', 'NS', 'QC', 'MB', 'ON', 'SK', 'QC', 'ON', 'AB',
       'QC', 'BC', 'NS', 'ON', 'MB', 'BC', 'NL', 'Q

In [57]:
bla = xs.search_data_catalogs(data_catalogs=dcat, variables_and_freqs={var: 'D'})
bla

2023-05-04 18:19:17 INFO     xscen.extract   Catalog opened: < catalog with 5 dataset(s) from 38 asset(s)> from 1 files.
2023-05-04 18:19:17 INFO     xscen.extract   Iterating over 4 potential datasets.
2023-05-04 18:19:17 INFO     xscen.extract   Found 2 with all variables requested and corresponding to the criteria.


{'GouvCan_AHCCD_CAN': < catalog with 1 dataset(s) from 1 asset(s)>,
 'GouvQc_info-climat-merged_QC': < catalog with 1 dataset(s) from 1 asset(s)>}

In [72]:
ds = ds_dict['D']
ds.cf['latitude']

<xarray.DataArray 'lat' (station: 463)>
dask.array<open_dataset-a1c9dfe793a7e3a85a53142cd8ec3ddclat, shape=(463,), dtype=float64, chunksize=(463,), chunktype=numpy.ndarray>
Coordinates:
  * station       (station) object '5030203' '7043540' ... '5062922' '8502591'
    stnid         (station) object dask.array<chunksize=(463,), meta=np.ndarray>
    station_name  (station) object dask.array<chunksize=(463,), meta=np.ndarray>
Attributes:
    units:      degrees_north
    long_name:  latitude

In [73]:
ds2 = ds.reset_coords(['lat', 'lon'])
ds2

<xarray.Dataset>
Dimensions:       (time: 65014, station: 463)
Coordinates:
  * time          (time) datetime64[ns] 1840-01-01 1840-01-02 ... 2017-12-31
  * station       (station) object '5030203' '7043540' ... '5062922' '8502591'
    station_name  (station) object dask.array<chunksize=(463,), meta=np.ndarray>
    stnid         (station) object dask.array<chunksize=(463,), meta=np.ndarray>
    elev          (station) int32 dask.array<chunksize=(463,), meta=np.ndarray>
    prov          (station) object dask.array<chunksize=(463,), meta=np.ndarray>
    fromyear      (station) int32 dask.array<chunksize=(463,), meta=np.ndarray>
    frommonth     (station) int32 dask.array<chunksize=(463,), meta=np.ndarray>
    toyear        (station) int32 dask.array<chunksize=(463,), meta=np.ndarray>
    tomonth       (station) int32 dask.array<chunksize=(463,), meta=np.ndarray>
    stns_joined   (station) object dask.array<chunksize=(463,), meta=np.ndarray>
Data variables:
    lon           (station) float64 dask.array<chunksize=(463,), meta=np.ndarray>
    lat           (station) float64 dask.array<chunksize=(463,), meta=np.ndarray>
    pr            (station, time) float64 dask.array<chunksize=(463, 65014), meta=np.ndarray>
Attributes: (12/26)
    title:                   Second Generation of Daily Adjusted Precipitatio...
    history:                 2022-09-27 : Convert from original format to NetCDF
    type:                    station_obs
    institute:               Environment and Climate Change Canada
    institute_id:            ECCC
    dataset_id:              AHCCD_gen2_day_precipitation
    ...                      ...
    cat:date_start:          1840-01-01 00:00
    cat:date_end:            2017-12-31 00:00
    cat:version:             gen2
    cat:_data_format_:       nc
    cat:path:                /tank/scenario/datasets/station-obs/GouvCan/AHCC...
    intake_esm_dataset_key:  GouvCan_AHCCD_CAN.CAN.raw.D

In [74]:
exp = ds.expand_dims(dim=['der_hengst'])
exp

<xarray.Dataset>
Dimensions:       (time: 65014, station: 463, der_hengst: 1)
Coordinates: (12/13)
  * time          (time) datetime64[ns] 1840-01-01 1840-01-02 ... 2017-12-31
  * station       (station) object '5030203' '7043540' ... '5062922' '8502591'
    station_name  (station) object dask.array<chunksize=(463,), meta=np.ndarray>
    stnid         (station) object dask.array<chunksize=(463,), meta=np.ndarray>
    lon           (station) float64 dask.array<chunksize=(463,), meta=np.ndarray>
    lat           (station) float64 dask.array<chunksize=(463,), meta=np.ndarray>
    ...            ...
    prov          (station) object dask.array<chunksize=(463,), meta=np.ndarray>
    fromyear      (station) int32 dask.array<chunksize=(463,), meta=np.ndarray>
    frommonth     (station) int32 dask.array<chunksize=(463,), meta=np.ndarray>
    toyear        (station) int32 dask.array<chunksize=(463,), meta=np.ndarray>
    tomonth       (station) int32 dask.array<chunksize=(463,), meta=np.ndarray>
    stns_joined   (station) object dask.array<chunksize=(463,), meta=np.ndarray>
Dimensions without coordinates: der_hengst
Data variables:
    pr            (der_hengst, station, time) float64 dask.array<chunksize=(1, 463, 65014), meta=np.ndarray>
Attributes: (12/26)
    title:                   Second Generation of Daily Adjusted Precipitatio...
    history:                 2022-09-27 : Convert from original format to NetCDF
    type:                    station_obs
    institute:               Environment and Climate Change Canada
    institute_id:            ECCC
    dataset_id:              AHCCD_gen2_day_precipitation
    ...                      ...
    cat:date_start:          1840-01-01 00:00
    cat:date_end:            2017-12-31 00:00
    cat:version:             gen2
    cat:_data_format_:       nc
    cat:path:                /tank/scenario/datasets/station-obs/GouvCan/AHCC...
    intake_esm_dataset_key:  GouvCan_AHCCD_CAN.CAN.raw.D

In [61]:
type(ds)

xarray.core.dataset.Dataset

<xarray.DataArray (lat: 463)>
array([52.35973667, 49.3       , 49.11666667, 55.2       , 51.30091667,
       53.25388889, 53.88333333, 44.56666667, 65.75529167, 50.46666667,
       49.01666667, 48.8       , 55.83333333, 55.19666722, 47.34638889,
       49.2       , 43.91835167, 52.18500056, 58.18333333, 49.995835  ,
       61.05      , 45.05      , 47.7       , 58.83638889, 46.75      ,
       44.56666667, 53.06916667, 51.26666667, 50.72222778, 49.71666667,
       44.24390333, 48.73472222, 50.36666667, 52.76666667, 45.76666667,
       67.40777778, 46.96666667, 68.81666667, 58.46666667, 49.15      ,
       47.3       , 46.53333333, 50.06666667, 55.28333333, 62.46277778,
       47.96666667, 58.76666667, 49.08222222, 54.8       , 60.17413889,
       45.23333333, 49.58388889, 44.75      , 52.44889056, 49.91666667,
       52.12902778, 49.35722222, 50.33168056, 45.31805556, 46.81666667,
       49.12695556, 52.08333333, 70.48611111, 62.20194444, 45.21639083,
       48.81666667, 44.66666667, 49.41388889, 50.78223333, 68.31666667,
       74.71694444, 49.46722778, 44.25      , 42.58333333, 44.58333333,
       53.58027972, 48.23333333, 59.56666667, 45.38333333, 49.18763889,
       48.56972222, 72.98333333, 57.25      , 46.4       , 68.75      ,
       49.37001389, 50.11666667, 50.58333333, 45.3       , 49.61666667,
       48.94638889, 50.97861278, 51.98333333, 62.81666667, 49.61222222,
       54.02261111, 51.51666667, 66.58333333, 51.19335833, 55.93611111,
...
       52.41666667, 52.11497222, 68.54055556, 51.10038889, 48.75973889,
       48.36666667, 49.23333333, 43.83083333, 50.67333333, 50.25694444,
       66.24083333, 46.56666667, 61.24194444, 48.65      , 50.63333333,
       49.18333333, 46.48333333, 50.07138889, 48.77694444, 47.98333333,
       79.98333333, 44.7       , 48.64694722, 49.81666667, 52.35974   ,
       60.7095    , 49.76666667, 50.38583611, 50.11416778, 44.6       ,
       51.06694444, 82.5       , 49.62388889, 49.1       , 46.62555556,
       46.03333333, 49.05      , 58.428335  , 46.56666667, 51.06666667,
       61.98333333, 46.06666667, 63.61666667, 49.82397222, 47.62222222,
       50.43333333, 59.56666667, 50.25      , 53.89083333, 69.10805556,
       45.88333333, 49.48888889, 53.15805583, 64.29888889, 42.45      ,
       46.95      , 45.4       , 50.68027778, 49.85722222, 47.91666667,
       52.16666667, 58.73333333, 51.29833333, 54.13222278, 50.05      ,
       43.67722222, 49.87611111, 50.55111111, 50.01611111, 67.81666667,
       50.26436111, 47.61666667, 54.82472222, 45.65      , 49.47833333,
       47.41666667, 45.01557833, 48.51666667, 48.53333333, 49.41666667,
       50.88333333, 68.65      , 48.647225  , 47.38333333, 46.1       ,
       56.08333333, 49.79027917, 49.55      , 62.1       , 49.21666667,
       47.00946944, 50.30564611, 52.42139056, 49.08333333, 45.03224833,
       49.02616667, 55.80333333, 52.30361111])
Coordinates:
  * lat      (lat) float64 52.36 49.3 49.12 55.2 51.3 ... 45.03 49.03 55.8 52.3

In [78]:
ds = ds.assign_coords(lat=xr.DataArray(data=ds.lat.values,
                   dims=["lat"],
                   coords=dict(lat=("lat", ds.lat.values))))
ds = ds.assign_coords(lon=xr.DataArray(data=ds.lon.values,
                   dims=["lon"],
                   coords=dict(lon=("lon", ds.lon.values))))
ds

<xarray.Dataset>
Dimensions:       (time: 65014, station: 463, lon: 463, lat: 463)
Coordinates: (12/13)
  * time          (time) datetime64[ns] 1840-01-01 1840-01-02 ... 2017-12-31
  * station       (station) object '5030203' '7043540' ... '5062922' '8502591'
    station_name  (station) object dask.array<chunksize=(463,), meta=np.ndarray>
    stnid         (station) object dask.array<chunksize=(463,), meta=np.ndarray>
  * lon           (lon) float64 -97.02 -69.55 -68.38 ... -118.5 -97.86 -55.83
  * lat           (lat) float64 52.36 49.3 49.12 55.2 ... 45.03 49.03 55.8 52.3
    ...            ...
    prov          (station) object dask.array<chunksize=(463,), meta=np.ndarray>
    fromyear      (station) int32 dask.array<chunksize=(463,), meta=np.ndarray>
    frommonth     (station) int32 dask.array<chunksize=(463,), meta=np.ndarray>
    toyear        (station) int32 dask.array<chunksize=(463,), meta=np.ndarray>
    tomonth       (station) int32 dask.array<chunksize=(463,), meta=np.ndarray>
    stns_joined   (station) object dask.array<chunksize=(463,), meta=np.ndarray>
Data variables:
    pr            (station, time) float64 dask.array<chunksize=(463, 65014), meta=np.ndarray>
Attributes: (12/26)
    title:                   Second Generation of Daily Adjusted Precipitatio...
    history:                 2022-09-27 : Convert from original format to NetCDF
    type:                    station_obs
    institute:               Environment and Climate Change Canada
    institute_id:            ECCC
    dataset_id:              AHCCD_gen2_day_precipitation
    ...                      ...
    cat:date_start:          1840-01-01 00:00
    cat:date_end:            2017-12-31 00:00
    cat:version:             gen2
    cat:_data_format_:       nc
    cat:path:                /tank/scenario/datasets/station-obs/GouvCan/AHCC...
    intake_esm_dataset_key:  GouvCan_AHCCD_CAN.CAN.raw.D

In [82]:
shape_file = "/scen3/braun/data/obs_synthese23/gis/lpr_000b16a_e_QC_simpl1.zip!lpr_000b16a_e_QC_simpl1.shp"
ds_qc = subset.subset_shape(ds.sortby(['lat', 'lon']), shape=shape_file)
ds_qc

InvalidIndexError: Reindexing only valid with uniquely valued Index objects